In [3]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import glob
import math


In [4]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'

In [5]:
os.listdir(File_path)[0:2]

['Ocean4_COM_MITgcm_JPL.nc', 'Ocean1_TYP_FVCOM.nc']

In [6]:
Data_Name = '*COM_NEMO-CNRS.nc'
paths = glob.glob(File_path + Data_Name)

In [7]:
paths

['/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']

Dans NN, var en x pour simul y<br>
x : T, S, x, y, z<br>
y : MeltRate<br>
1ere Étape : Preparer les données

In [106]:
def get_depth_temp_sali(df, Data, zs):
    ys, IceDraft = df.ny, df.iceDraft
    N_T, N_S = [], []
    ylast = None
    dataset = Data[['temperatureYZ', 'salinityYZ']].to_dataframe().dropna()
    Index_Y = dataset.index.get_level_values('ny')
    for ind, y in enumerate(ys):
        if (ind+1)%int(len(ys)/10) == 0:
            print('Starting {}/{}'.format(ind+1, len(ys)), end='\r')
        indz = np.abs(np.array(zs + IceDraft[ind])).argmin()
        if y != ylast:
            Slice = dataset.iloc[Index_Y == y].droplevel('ny')
        Z_N = abs(np.array(Slice.index - indz)).argmin() + Slice.index[0]
        New_T = dataset.temperatureYZ.loc[Z_N, y]
        New_S = dataset.salinityYZ.loc[Z_N, y]
        N_T.append(New_T)
        N_S.append(New_S)
    return np.array(N_T), np.array(N_S)

In [112]:
def New_get_depth_temp_sal(df, Data):
    ys, Icedraft = np.array(df.ny), np.array(df.iceDraft)
    Var = ['temperatureYZ', 'salinityYZ']
    N_T, N_S = [], []
    Cur = Data[Var].sel(ny=ys).dropna('nz').interp({'nz': - Icedraft})#.dropna('nz')#.interp({'nz': -4.})
    T, S = np.array(Cur.to_array())
    return Cur


path = paths[0]
dfT = pd.DataFrame()
Name = path.replace('.nc', '').replace(File_path, '')
Data = xr.open_dataset(path)

t = 0
Data = Data.assign_coords({'nx': Data.x,
                           'ny': Data.y,
                           'nz': Data.z,
                           'nTime': Data.time})
df = pd.DataFrame()
df['meltRate'] = Data['meltRate'].isel(nTime = 0).to_dataframe()['meltRate']

df['iceDraft'] = Data['iceDraft'].to_dataframe()['iceDraft']

df['date'] = t
df.dropna(inplace = True)
df.reset_index(inplace=True)
Cur = New_get_depth_temp_sal(df, Data.isel(nTime = 0))

In [118]:
ys, Icedraft = np.array(df.ny), np.array(df.iceDraft)
Var = ['temperatureYZ', 'salinityYZ']
ys

array([ 1000.,  1000.,  3000., ..., 77000., 79000., 79000.])

In [119]:
Cur = Data[Var].sel(ny=ys).dropna('nz').interp({'nz': - Icedraft})

KeyError: -596.0

In [ ]:
Cur = Data[Var].sel(ny=ys).dropna('nz').interp({'nz': - 600})

In [113]:
for path in paths:
    dfT = pd.DataFrame()
    Name = path.replace('.nc', '').replace(File_path, '')
    print(path)
    Data = xr.open_dataset(path)
    Data = Data.assign_coords({'nx': Data.x,
                           'ny': Data.y,
                           'nz': Data.z,
                           'nTime': Data.time})
    T = np.array(Data.nTime.as_numpy())
    Horiz_resol = 2 #km/index
    XstartYZ = int(500 / Horiz_resol)
    YstartXZ = int(40 / Horiz_resol)
    time = len(T)

    for t in range(time):
        if (t+1)%int(time/10) == 0:
            print('Starting {} / {}'.format(t+1, time))
        df = pd.DataFrame()
        df['meltRate'] = (Data['meltRate'].isel(nTime = t).to_dataframe())['meltRate']
        df['thermalDriving'] = Data['thermalDriving'].isel(nTime = t).to_dataframe()['thermalDriving']
        df['halineDriving'] = Data['halineDriving'].isel(nTime = t).to_dataframe()['halineDriving']
        if Data['iceDraft'].ndim == 2:
            df['iceDraft'] = Data['iceDraft'].to_dataframe()['iceDraft']
        else:
            df['iceDraft'] = Data['iceDraft'].isel(nTime = t).to_dataframe()['iceDraft']
        df['date'] = t
        df.dropna(inplace = True)
        df.reset_index(inplace=True)
        T, S = New_get_depth_temp_sal(df, Data.isel(nTime = t))
        #T, S = get_depth_temp_sali(df, Data.isel(nTime = t), Data.z.to_dataframe())
        df['temperatureYZ'] = T
        df['salinityYZ'] = S
        dfT = pd.concat([dfT, df], ignore_index = True)
    dfT.dropna(inplace = True)
    dfT.reset_index(drop = True, inplace = True)
    dfT.rename(columns={'nx' :'x', 'ny': 'y'}, inplace=True)
    dfT['x'] = dfT['x'] * Horiz_resol
    dfT['y'] = dfT['y'] * Horiz_resol
    pd.DataFrame.to_csv(dfT, Bet_path + 'Data/data_{}.csv'.format(Name), index = False)

/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc


KeyError: -596.0

In [115]:
Data.interp({'nz' : -596.0})

<xarray.Dataset>
Dimensions:                    (nTime: 240, ny: 40, nx: 240)
Coordinates:
  * nx                         (nx) float32 3.21e+05 3.23e+05 ... 7.99e+05
  * ny                         (ny) float32 1e+03 3e+03 ... 7.7e+04 7.9e+04
  * nTime                      (nTime) object 0000-01-01 00:00:00 ... 0019-12...
    nz                         float64 -596.0
Data variables: (12/25)
    barotropicStreamfunction   (nTime, ny, nx) float32 -0.0 -0.0 ... 1.443e+04
    salinityYZ                 (nTime, ny) float64 nan nan nan ... nan nan nan
    temperatureYZ              (nTime, ny) float64 nan nan nan ... nan nan nan
    salinityXZ                 (nTime, nx) float64 nan nan nan ... 34.54 34.53
    temperatureXZ              (nTime, nx) float64 nan nan nan ... 0.4822 0.46
    bottomSalinity             (nTime, ny, nx) float32 nan nan ... 34.69 34.69
    ...                         ...
    time                       (nTime) object 0000-01-01 00:00:00 ... 0019-12...
    uBoundaryLayer             (nTime, ny, nx) float32 nan nan nan ... nan nan
    vBoundaryLayer             (nTime, ny, nx) float32 nan nan nan ... nan nan
    x                          (nx) float32 3.21e+05 3.23e+05 ... 7.99e+05
    y                          (ny) float32 1e+03 3e+03 ... 7.7e+04 7.9e+04
    z                          float64 -596.0
Attributes: (12/19)
    experiment:                      ISOMIP+ COM_EXP1
    contact:                         nicolas.jourdain@univ-grenoble-alpes.fr
    institute:                       IGE-CNRS, Grenoble, France
    model:                           NEMO_3.6 (trunk r6402)
    repository:                      http://www.nemo-ocean.eu
    vertical_coordinates:            Z* (variable-volume levels with partial ...
    ...                              ...
    equation_of_state:               linear
    GammaT:                          0.026
    GammaS:                          0.000742857
    Cd:                              0.0025
    original_name:                   EXP1publi
    version:                         1.1 (bug on surface freshwater correctio...